# Exploring Connect 4 with NeuroEvolution

Inside this notebook I will test the NEAT (Neuroevolution of augmenting topologies) algorithm on a connect 4 environment.
Why connect 4? Because it's a solved game. In the 1988, James D. Allen was able to find a way to solve connect 4. But it wasn't until 1995, when John Tromp was able to brute force every position. 

**I won't try to brute force it (again), but test if a NeuralNetwork trained with Evolution, is able to find the solving strategy.**

Before jumping to the NEAT part, I decide to write the Connect 4 environment and the agents that will play (random, greedy and minmax). It would be easier to search a pre-made code online, but I prefer to have the full frame of my workbench.

## Connect 4 environment

Let's start by importing all the necessaries librarys

In [19]:
from random import choice, randint
from copy import deepcopy

import numpy as np

### Board Class Implementation

The Board class is the core data structure of our Connect 4 implementation. It efficiently represents the game state with three main components:

- A 2D matrix representing the board (using NumPy arrays for performance)
- A tracking system for available columns where pieces can be placed
- A "gravity position" mechanism that simulates how pieces fall to the lowest available slot

The class provides methods to make moves, visualize the board with colored output, and create deep copies of the game state. All implementation details are thoroughly documented within the class code.

In [20]:
# CONSTANTS 
ROWS = 6 # number of rows in the Connect 4 board
COLS = 7 # number of columns in the Connect 4 board
X = 1 # player 1 (this allow a easy swapping of players by just changing the sign)
Y = -1 # player 2

class Board:

    def __init__(self):
        ''' This function initializes the board with the following attributes:
            - self.matrix: a 2D numpy array representing the game board, initialized to zeros. Rows and columns are swapped for cache efficiency.
            - self.availableMoves: a 1D numpy array containing the indices of columns that are not full.
            - self.gravityPosition: a 1D numpy array indicating the next available row in each column where a piece can be placed.

            How the Board class is initialized:
            Matrix: (ROWS x COLS)
                0 0 0 0 0 0 <- column 0
                0 0 0 0 0 0 <- column 1
                0 0 0 0 0 0 <- column 2
                0 0 0 0 0 0 <- column 3
                0 0 0 0 0 0 <- column 4
                0 0 0 0 0 0 <- column 5
                0 0 0 0 0 0 <- column 6
                ^ ^ ^ ^ ^ ^
                | | | | | |
                | | | | | +-- row 0
                | | | | +---- row 1
                | | | +------ row 2
                | | +-------- row 3
                | +---------- row 4
                +------------ row 5

            AvailableMoves: [0 1 2 3 4 5 6]
            GravityPosition: [5 5 5 5 5 5 5]

            When a player makes a move in column 3, the board updates as follows:
            Matrix: (ROWS x COLS) AvailableMoves: [0 1 2 3 4 5 6] GravityPosition: [5 5 5 4 5 5 5]
                0 0 0 0 0 0                                                               ^
                0 0 0 0 0 0                                                               |                                   
                0 0 0 0 0 0                     I moved the index of column 3 down by 1 --+ 
                0 0 0 0 0 1 
                0 0 0 0 0 0 
                0 0 0 0 0 0 
                0 0 0 0 0 0 

            '''
        self.matrix = np.zeros((COLS, ROWS), dtype=int)
        self.availableMoves = np.array([i for i in range(COLS)])
        self.gravityPosition = np.array([ROWS - 1 for _ in range(COLS)])


    def makeMove(self, column_to_insert, by_player) -> None:
        ''' This function inserts a player's move into the board at the specified column.
            It updates the available moves and gravity position accordingly.

            Input:
                move: int, the column index where the player wants to place their piece.
                player: int, either X (1) or Y (-1), representing the player making the move.

            Output:
                None (modifies self.matrix, self.availableMoves and self.gravityPosition in place)
        '''    
        # Because of the way the board is represented, we need to swap rows and columns
        self.matrix[column_to_insert, self.gravityPosition[column_to_insert]] = by_player
        # Reduce the gravity position for that column
        self.gravityPosition[column_to_insert] -= 1

        # If the gravity position is -1, it means that the last available row in that column was just filled (0)
        if self.gravityPosition[column_to_insert] == -1:
            # I simply remove the column from availableMoves
            self.availableMoves = np.delete(self.availableMoves, np.where(self.availableMoves == column_to_insert)[0])

        return

    def printBoard(self) -> None:
        ''' This function prints the board in a human-readable format.
            X is represented by 1, O by -1 and empty cells by 0.

            Input:
                None (uses self.matrix)

            Output:
                None (prints the board to the console)
        '''

        # ANSI color codes for colored output
        RED = '\033[31m'
        YELLOW = '\033[33m'
        BLUE = '\033[34m'
        RESET = '\033[0m'

        for r in range(self.matrix.shape[1]):
            row = self.matrix.T[r]
            s = []
            for v in row:
                if v == X:
                    s.append(f"{BLUE}| {RED}X{BLUE} |{RESET}")
                elif v == Y:
                    s.append(f"{BLUE}| {YELLOW}O{BLUE} |{RESET}")
                else:
                    s.append(f"{BLUE}| {BLUE}.{BLUE} |{RESET}")
            print("".join(s))

        for c in range(COLS):
            print(f"{BLUE}| {c} |{RESET}", end="")
        print("\n")

        return None
    
    def children(self) -> tuple:
        ''' This function returns a deep copy of the current board state, including the matrix, available moves, and gravity positions.

            Input:
                None (uses self.matrix, self.availableMoves, self.gravityPosition)
            Output:
                tuple: (matrix, availableMoves, gravityPosition)
        '''
        return deepcopy(self.matrix), deepcopy(self.availableMoves), deepcopy(self.gravityPosition)
    

### Win Detection Algorithm

Next, I implement an optimized win-checking function. Rather than scanning the entire board after each move (which would be O(ROWS × COLS)), this algorithm focuses only on checking lines that pass through the last placed piece.

This targeted approach offers significant efficiency benefits:
- For moves near the board edges, fewer positions need to be examined
- We only check the four possible winning directions (vertical, horizontal, and two diagonals)
- In the worst case, the algorithm approaches the performance of a full board scan, but on average performs much better

The function handles both Board objects and raw matrix representations, allowing for flexible use in different contexts.

In [21]:
def checkWin(last_move, current_player, matrix, y=None) -> bool:
        ''' This function checks if there is a winner in the game.
            It checks all rows, columns and diagonals (close to last_move) for a sequence of 4 identical non-zero values.

            Input:
                last_move: the last move made (row index)
                current_player: the player that made the last move (X or Y)
                matrix: the game board (numpy 2D array) or a Board object
                y: the row index of the last move (optional, needed if matrix is a numpy array)

            Output:
                True if there is a winner, False otherwise.
        '''
        # Check if matrix is a Board object or a numpy array
        if type(matrix) is not np.ndarray: # If it's a Board object, extract the matrix and gravity positions
            matrix, _, y = matrix.children()
            y = y[last_move] + 1 
        elif y is None: # If it's a numpy array, y must be provided
            print("Warning: y coordinate not provided.")

        # Check vertical (column)
        if y <= ROWS - 4:  # Ensure there are at least 4 cells below
            if matrix[last_move, y] == matrix[last_move, y + 1] == matrix[last_move, y + 2] == matrix[last_move, y + 3] == current_player:
                return True
        
        # Check horizontal (row)
        start = max(0, last_move - 3) # Clip to board bounds
        end = min(COLS - 1, last_move + 3) # Clip to board bounds
        line = [matrix[c, y] for c in range(start, end + 1)] # Extract the row segment

        if len(line) >= 4: # Ensure there are at least 4 cells in the line
            for i in range(0, len(line) - 3):
                if (line[i] == line[i+1] == line[i+2] == line[i+3] == current_player):
                    return True

        # Check diagonal (top-left to bottom-right) and (top-right to bottom-left)
        # Build diagonal sequences centered at (x,y) using offsets -3..+3 and clip to board bounds.
        
        # Diagonal 1: top-left to bottom-right -> positions (x + i, y + i)
        diag1 = []
        for i in range(-3, 4):
            nx = last_move + i
            ny = y + i
            if 0 <= nx < COLS and 0 <= ny < ROWS:
                diag1.append(matrix[nx, ny])

        if len(diag1) >= 4:
            for i in range(0, len(diag1) - 3):
                if diag1[i] == diag1[i+1] == diag1[i+2] == diag1[i+3] == current_player:
                    return True

        # Diagonal 2: top-right to bottom-left -> positions (x + i, y - i)
        diag2 = []
        for i in range(-3, 4):
            nx = last_move + i
            ny = y - i
            if 0 <= nx < COLS and 0 <= ny < ROWS:
                diag2.append(matrix[nx, ny])

        if len(diag2) >= 4:
            for i in range(0, len(diag2) - 3):
                if diag2[i] == diag2[i+1] == diag2[i+2] == diag2[i+3] == current_player:
                    return True

        return False

### Game Orchestration

The Game class serves as the controller for Connect 4 matches, handling:

- Game initialization with different board types and player agents
- Turn management, alternating between players
- Move validation and application to the board
- Win condition checking after each move
- Game termination logic (win, draw, or invalid move)
- Results reporting with statistics

This class makes it easy to test different agents against each other by simply passing different player functions to the constructor. The `simulateLocalGame()` method provides a complete simulation with visual board rendering after each move, creating a complete game loop from start to finish.

In [22]:
class Game:

    def __init__(self, startBoard, player1, player2):
        ''' This class manages a Connect 4 game between two players.

            Input:
                startBoard: the initial game board (Board object)
                player1: the first player algorithm (function)
                player2: the second player algorithm (function)
        '''
        self.board = startBoard
        self.player1 = player1
        self.player2 = player2

    def simulateLocalGame(self, print_board=True) -> dict:
        ''' This function simulates a Connect 4 game between two players.

            Input:
                print_board: bool, if True prints the board after each move (default True)
            
            Output:
                dict: {'winner': 'X' or 'O' or 'Draw', 'moves': number of moves made}
        '''
        board = Board()
        isPlayer1 = True
        move_count = 0

        if print_board:
            print("=== Ready...GAME ===")
            board.printBoard()
        
        while len(board.availableMoves) > 0:
            current_player_value = 1 if isPlayer1 else -1

            if isPlayer1:
                move = self.player1(board, current_player_value)
            else:
                move = self.player2(board, current_player_value)

            if move not in board.availableMoves:
                print(f"ERROR: Non valid move: {move}")
                break

            move_count += 1
            if print_board:
               print(f"Move #{move_count}: Player {'X' if isPlayer1 else 'O'} choose column {move}")
            board.makeMove(move, current_player_value)
            if print_board:
                board.printBoard()

            if checkWin(move, current_player_value, board):
                print("Player", "1" if isPlayer1 else "2", "wins!")
                return {'winner': 'X' if isPlayer1 else 'O', 'moves': move_count}
            
            isPlayer1 = not isPlayer1

        print("🤝 Draw!")
        return {'winner': 'Draw', 'moves': move_count}

## Agents

Now, I can move on with some agent that will play the game. As I already said, I decide to implement a random agent, greedy and minmax.

### Random Agent
So, here the code for the random, that simply choose between all available moves with equal probability.

In [23]:
def random_agent(board, player=None) -> int:
    _, available_moves, _ = board.children()
    return available_moves[randint(0, len(available_moves) - 1)] if len(available_moves) > 0 else -1

game = Game(Board, random_agent, random_agent)
game.simulateLocalGame()

=== Ready...GAME ===
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move #1: Player X choose column 3
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || X || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move #2: Player O choose column 4
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || X || O || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move #3: Player X choose column 0
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . |

{'winner': 'O', 'moves': 14}

### Greedy Agent

At this point, the whole Connect 4 environment works correctly, and we've seen two random players compete against each other.

The next step is implementing a more advanced algorithm that can better compete with the neural networks we'll evolve later. For this, I'll create a greedy agent that evaluates possible moves and selects the best one.

To evaluate board positions effectively, I've developed a heuristic approach based on two key factors:

1. **Pattern recognition** - Detecting how many friendly pieces are connected or could form winning patterns
2. **Strategic positioning** - Evaluating if a piece is placed in a strategically valuable location

The first factor is implemented by scanning the board for potential two-piece and three-piece connections that could lead to wins. The second factor uses a position-based heatmap that assigns higher values to central columns (where more winning combinations intersect) and lower values to edge columns.

This combination of pattern detection and positional strategy gives our greedy agent a solid foundation for making intelligent moves without requiring deep search.

In [24]:
# Heuristic heatmap
heatmap = np.array(
                    [1, 2, 3, 4, 3, 2, 1]
                 )

def countPieceColumn(column, player) -> int:
    """Count the pieces of a player in a specific column"""
    return np.sum(column == player)

def contiguous_count(board, player) -> dict:
    ''' This function counts the number of contiguous pieces (2 or 3 in a row) for a given player in all directions.

        Input:
            board: the current game board (Board object)
            player: the player to check (1 or -1)

        Output:
            dict: {'two_piece': count of two contiguous pieces, 'three_piece': count of three contiguous pieces}
    '''
    matrix, _, gravity = board.children() # Extract the matrix from the board
    rows, cols = matrix.shape 

    results = {'two_piece': 0, 'three_piece': 0}

    # 4 pole to check: horizontal, vertical, diagonals
    directions = [
        (0, 1),   # horizontal
        (1, 0),   # vertical
        (1, 1),   # main diagonal
        (1, -1)   # secondary diagonal
    ]

    for dx, dy in directions:
        for x in range(rows):
            for y in range(cols):
                # Check sequences of length 4 starting from (x,y)
                sequences = []
                for i in range(4):
                    nx, ny = x + i*dx, y + i*dy
                    if 0 <= nx < rows and 0 <= ny < cols:
                        sequences.append(matrix[nx, ny])
                    else:
                        break

                if len(sequences) == 4:
                    # Count pieces of the player and empty cells
                    player_piece = sequences.count(player)
                    empty_cells = sequences.count(0)

                    # Two consecutive pieces with space to expand
                    if player_piece == 2 and empty_cells == 2:
                        results['two_piece'] += 1
                    # Three consecutive pieces with space to expand
                    elif player_piece == 3 and empty_cells == 1:
                        results['three_piece'] += 1

    return results

def evaluatePositions(board, move, current_player) -> int:
    ''' This function evaluates the board position after a potential move for the current player.
        
        Input:
            board: the current game board (Board object)
            move: the column index where the player wants to place their piece (int)
            current_player: the player making the move (1 or -1)
    '''

    opponents_player = -1 if current_player == 1 else 1

    # Take a snapshot of the board to simulate the move without affecting the original board
    simulation_board = deepcopy(board)

    # Emulate the move
    simulation_board.makeMove(move, current_player)

    # Check for victory after the simulated move
    if checkWin(move, current_player, simulation_board):
        return 1000

    # Emulate what happens if the opponent plays in the same column
    matrix, _, y = simulation_board.children()
    y = y[move] + 1
    matrix[move,y] = opponents_player

    # Check if the opponent would win with this move
    if checkWin(move, opponents_player, matrix, y):
        return 500

    # Check for draw
    if len(simulation_board.availableMoves) == 0:  # board is full
        return 0

    # 2. Count points for almost-alignments and center control
    tot_score = 0

    # Weights to calibrate
    two_pieces_weight = 5
    three_pieces_weight = 20
    
    player_heat = 0
    opponent_heat = 0

    matrix, _, _ = simulation_board.children()
    # a. Check heat in columns
    for i, column in enumerate(matrix):  # Iterate over columns
        player_heat += countPieceColumn(column, current_player) * heatmap[i]
        opponent_heat += countPieceColumn(column, opponents_player) * heatmap[i]
    tot_score += player_heat - opponent_heat

    # b. Count threats (two or three pieces in a row)
    player_contiguous = contiguous_count(simulation_board, current_player)
    opponent_contiguous = contiguous_count(simulation_board, opponents_player)

    tot_score += (player_contiguous['three_piece'] * three_pieces_weight)
    tot_score -= (opponent_contiguous['three_piece'] * three_pieces_weight)

    tot_score += (player_contiguous['two_piece'] * two_pieces_weight)
    tot_score -= (opponent_contiguous['two_piece'] * two_pieces_weight)

    return tot_score

def greedy_agent(board, player) -> int:
    ''' This function selects the best move for the current player based on a heuristic evaluation of the board.
        If multiple moves have the same best score, it randomly selects one of them. This prevents predictability.

        Input:
            board: the current game board (Board object)
            player: the player making the move (1 or -1)

        Output:
            int: the column index of the best move
    '''
    available_moves = board.availableMoves
    if len(available_moves) == 0:
        return -1  # No moves available

    best_move = None
    best_score = -float('inf')
    best_moves = [] 

    # Evaluate each available move
    for move in available_moves: 
        score = evaluatePositions(board, move, player)

        # Track the best score and corresponding moves
        if score > best_score:
            best_score = score
            best_move = move
        elif score == best_score: # If multiple moves have the same best score, track them
            best_moves.append((move, score))

    i = len(best_moves) - 1
    while i >= 0: # Remove moves that are not on the same level of best score
        if best_moves[i][1] < best_score:
            best_moves.remove(best_moves[i])
        i -= 1
        
    # Randomly choose among the best moves if there are multiple
    if best_moves: 
        best_moves.append((best_move, best_score))  # Include the initially found best move
        best_move, best_score = choice(best_moves)  # Randomly choose among the best moves

    return best_move

In [25]:
game2 = Game(Board, greedy_agent, greedy_agent)
game2.simulateLocalGame()

=== Ready...GAME ===
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move #1: Player X choose column 3
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || X || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move #2: Player O choose column 3
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || O || . || . || . |
| . || . || . || X || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move #3: Player X choose column 2
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . |

{'winner': 'O', 'moves': 14}

### Minimax Agent with Alpha-Beta Pruning

After implementing the greedy agent, which makes decisions based on immediate board evaluation, I'll now create a more sophisticated Minimax agent that can look ahead multiple moves.

The Minimax algorithm is a recursive decision-making approach that:

1. **Simulates future game states** - It builds a tree of possible future positions by playing moves and counter-moves
2. **Evaluates terminal positions** - At a specified depth or when the game ends, it evaluates the position
3. **Propagates values backward** - The algorithm assumes optimal play from both players, with the maximizer selecting the highest values and the minimizer selecting the lowest

To improve efficiency, I've implemented **Alpha-Beta pruning** which eliminates branches that cannot influence the final decision, dramatically reducing the search space without affecting the result.

My implementation includes several key components:
- A terminal state detector to identify game-ending positions
- A static evaluation function similar to our greedy approach
- The recursive Minimax algorithm with pruning
- A variable depth parameter to control the search complexity

By looking ahead multiple moves, this agent should make more strategic decisions than the greedy agent, particularly in complex tactical situations where immediate evaluation is insufficient.

In [26]:
# The function is almos the same as evaluatePositions but it doesn't simulate a move, it just evaluates the current board state.
def evaluate_board_state(board, player) -> int:
    """ Evaluate the board state for the given player using a heuristic approach. 

        Input:
            board: the current game board (Board object)
            player: the player for whom to evaluate the board (1 or -1)
    """
    opponents_player = -player
    
    tot_score = 0
    two_pieces_weight = 5  
    three_pieces_weight = 20
    
    player_heat = 0
    opponent_heat = 0

    matrix, _, _ = board.children()
    
    for i, column in enumerate(matrix):
        player_heat += countPieceColumn(column, player) * heatmap[i]
        opponent_heat += countPieceColumn(column, opponents_player) * heatmap[i]
    tot_score += player_heat - opponent_heat

    # Conteggio delle Minacce (due o tre pedine in fila)
    player_contiguous = contiguous_count(board, player)
    opponent_contiguous = contiguous_count(board, opponents_player)

    tot_score += (player_contiguous['three_piece'] * three_pieces_weight)
    tot_score -= (opponent_contiguous['three_piece'] * three_pieces_weight)

    tot_score += (player_contiguous['two_piece'] * two_pieces_weight)
    tot_score -= (opponent_contiguous['two_piece'] * two_pieces_weight)

    return tot_score

def minimax(board, depth, alpha, beta, is_maximizing, maximizing_player) -> int:
    """
    Minimax algorithm with CORRECT Alpha-Beta pruning for Connect 4.
    
    Input:
        board: the current game board (Board object)
        depth: the maximum depth to search in the game tree (int)
        alpha: the best score that the maximizer currently can guarantee at that level or above (int)
        beta: the best score that the minimizer currently can guarantee at that level or above (int)
        is_maximizing: boolean indicating if the current turn is for the maximizing player (True) or minimizing player (False)
        maximizing_player: the player who is maximizing the score (1 or -1)
    """
    
    # Base cases
    if len(board.availableMoves) == 0:
        return 0
    elif depth == 0:
        return evaluate_board_state(board, maximizing_player)
    
    current_player = maximizing_player if is_maximizing else -maximizing_player

    # Maximizing player's turn
    if is_maximizing:
        max_eval = -float('inf')
        
        for mossa in board.availableMoves:
            nuova_board = deepcopy(board)
            nuova_board.makeMove(mossa, current_player)
            
            # Check for immediate win
            if checkWin(mossa, current_player, nuova_board):
                return 10000 + depth 
            
            # Recursive call
            eval_score = minimax(nuova_board, depth-1, alpha, beta, False, maximizing_player)
            max_eval = max(max_eval, eval_score)
            
            # Alpha-Beta pruning
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break  # Beta cutoff
                
        return max_eval

    # Minimizing player's turn
    else:
        min_eval = float('inf')
        
        for mossa in board.availableMoves:
            nuova_board = deepcopy(board)
            nuova_board.makeMove(mossa, current_player)
            
            # Check for immediate win (opponent's win from our perspective)
            if checkWin(mossa, current_player, nuova_board):
                return -10000 - depth
            
            # Recursive call
            eval_score = minimax(nuova_board, depth-1, alpha, beta, True, maximizing_player)
            min_eval = min(min_eval, eval_score)
            
            # Alpha-Beta pruning
            beta = min(beta, eval_score)
            if beta <= alpha:
                break  # Alpha cutoff
                
        return min_eval

def minimax_agent(board, player, depth=4) -> int:
    """
    Minimax agent with CORRECT Alpha-Beta pruning for Connect 4.
    
    Input:
        board: the current game board (Board object)
        player: the player making the move (1 or -1)
        depth: the maximum depth to search in the game tree (int, default is 4)
        
    Output:
        int: the column index of the best move
    """
    if len(board.availableMoves) == 0:
        return -1
    
    best_moves = []
    best_score = -float('inf')
    
    for mossa in board.availableMoves:
        nuova_board = deepcopy(board)
        nuova_board.makeMove(mossa, player)

        # Check for immediate win
        if checkWin(mossa, player, nuova_board):
            return mossa
        
        # Get score using minimax with alpha-beta pruning
        # Initialize alpha and beta for each root move
        alpha = -float('inf')
        beta = float('inf')
        score = minimax(nuova_board, depth-1, alpha, beta, False, player)
        print(f"Move: {mossa}, Score: {score}", flush=True)
        
        if score > best_score:
            best_score = score
            best_move = mossa
        elif score == best_score:
            best_moves.append((mossa, score))

    i = len(best_moves) - 1
    while i >= 0:
        if best_moves[i][1] < best_score:
            best_moves.remove(best_moves[i])
        i -= 1

    if best_moves:
        best_moves.append((best_move, best_score))  # Include the initially found best move
        best_move, best_score = choice(best_moves)  # Randomly choose among the best moves

    return best_move
    

In [29]:
print("Minimax (X) vs Heuristic (O)")
game = Game(Board, minimax_agent, greedy_agent)
game.simulateLocalGame()

Minimax (X) vs Heuristic (O)
=== Ready...GAME ===
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move: 0, Score: -9
Move: 1, Score: -8
Move: 2, Score: 0
Move: 3, Score: 0
Move: 4, Score: 0
Move: 5, Score: -8
Move: 6, Score: -9
Move #1: Player X choose column 4
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || X || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move #2: Player O choose column 3
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || O || X || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 |

{'winner': 'X', 'moves': 25}

In [ ]:
print("Minimax (X) vs Minimax (O)")
game = Game(Board, minimax_agent, minimax_agent)
game.simulateLocalGame()

Minimax (X) vs Heuristic (O)
=== Ready...GAME ===
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move: 0, Score: -9


Move: 1, Score: -8
Move: 2, Score: 0
Move: 3, Score: 0
Move: 4, Score: 0
Move: 5, Score: -8
Move: 6, Score: -9
Move #1: Player X choose column 3
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || X || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move: 0, Score: -26
Move: 1, Score: -17
Move: 2, Score: -24
Move: 3, Score: -17
Move: 4, Score: -24
Move: 5, Score: -17
Move: 6, Score: -26
Move #2: Player O choose column 1
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || . || . || . || . || . || . |
| . || O || . || X || . || . || . |
| 0 || 1 || 2 || 3 || 4 || 5 || 6 |

Move: 0, Score: -6
Move: 1, Score: -1
Move: 2, Score: -2
Move: 3, Score: -2
Move: 4, Score: 7
Move: 5, Score: 5
Move: 6, Score: -1
Move #3: Player X choose column 4
| . || . |

{'winner': 'X', 'moves': 41}